In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install pandas
!pip install numpy
!pip install sentence-transformers
!pip install attacut
!pip install scikit-learn
!pip install faiss-cpu


In [2]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import attacut
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [17]:
# Cell 1: Initialize Models
print("Initializing models...")
# Initialize SentenceTransformer (ใช้โมเดลที่รองรับภาษาไทย)
model = SentenceTransformer('Qwen/Qwen3-Embedding-0.6B')

# Test AttaCut (no need to load model, it's automatic)
print("Testing AttaCut...")
test_tokens = attacut.tokenize("ทดสอบการตัดคำภาษาไทย")
print(f"AttaCut test result: {test_tokens}")

print("Models initialized successfully!")

Initializing models...
Testing AttaCut...
AttaCut test result: ['ทดสอบ', 'การ', 'ตัด', 'คำ', 'ภาษา', 'ไทย']
Models initialized successfully!


In [19]:
# Cell 2: Data Loading
def load_data(file_path='/content/drive/MyDrive/AlphaZero_Backups/data/36strategies-description.csv'):
    """
    เตรียมข้อมูล - อ่านไฟล์ CSV
    """
    try:
        df = pd.read_csv(file_path)
        print(f"Data loaded successfully! Shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        return df
    except FileNotFoundError:
        print(f"File {file_path} not found")
        return None

# Load data
df = load_data()
if df is not None:
    print("\nFirst few rows:")
    print(df.head())

Data loaded successfully! Shape: (36, 5)
Columns: ['ชื่อกลยุทธ์', 'หมวด', 'ลักษณะ', 'ตัวอย่างกลยุทธ์', 'ตัวอย่างการเดินหมาก']

First few rows:
        ชื่อกลยุทธ์    หมวด  \
0    ปิดฟ้าข้ามทะเล  ชนะศึก   
1  ล้อมเว่ยช่วยจ้าว  ชนะศึก   
2       ยืมดาบฆ่าคน  ชนะศึก   
3    รอซ้ำยามเปลี้ย  ชนะศึก   
4       ชิงซ้ำตามไฟ  ชนะศึก   

                                              ลักษณะ  \
0  กลยุทธ์นี้เกี่ยวข้องกับการทำสิ่งใดสิ่งหนึ่งอย่...   
1  กลยุทธ์นี้มีความหมายว่าศัตรูได้รวบรวมกำลังทหาร...   
2  หมายถึงการกำจัดศัตรูที่มีความเข้มแข็ง ไม่จำเป็...   
3  หมายถึงการที่ศัตรูยังคงมีความเข้มแข็ง กำลังไพร...   
4  หมายถึงการที่ศัตรูยังคงมีสภาพที่อ่อนแอและย่ำแย...   

                                     ตัวอย่างกลยุทธ์  \
0  การนำเอากลยุทธ์ "ปิดฟ้าข้ามทะเล" ถูกใช้โดยลกซุ...   
1  การนำเอา "กลยุทธ์ล้อมเว่ยช่วยจ้าว" ไปใช้ได้แก่...   
2  การนำเอากลยุทธ์ยืมดาบฆ่าคนไปใช้ได้แก่ จิวยี่ที...   
3  การนำเอากลยุทธ์รอช้าตามเปลี่ยนไปใช้ได้แก่ ลกซุ...   
4  การนำเอากลยุทธ์ชิงซ้ำตามไฟไปใช้ได้แก่ ตังโต๊ะท... 

In [20]:
# Cell 3: Semantic Chunking Functions
def split_sentences(text):
    """
    Split text into sentences using regex or AttaCut
    """
    # Thai sentence ending patterns
    sentence_endings = r'[.!?。๏\n]+'
    sentences = re.split(sentence_endings, text)
    sentences = [s.strip() for s in sentences if s.strip()]

    # If no sentences found, use AttaCut
    if len(sentences) <= 1:
        words = attacut.tokenize(text)
        # Group words into sentences (simple approach)
        sentences = []
        current_sentence = ""
        for word in words:
            current_sentence += word
            if len(current_sentence) > 100:  # Arbitrary length
                sentences.append(current_sentence.strip())
                current_sentence = ""
        if current_sentence:
            sentences.append(current_sentence.strip())

    return sentences

def compute_sentence_embeddings(sentences):
    """
    Compute embeddings for sentences
    """
    if not sentences:
        return np.array([])
    return model.encode(sentences)

def calculate_cosine_similarity(embeddings):
    """
    Calculate cosine similarity between consecutive embeddings
    """
    if len(embeddings) <= 1:
        return []

    similarities = []
    for i in range(len(embeddings) - 1):
        sim = np.dot(embeddings[i], embeddings[i+1]) / (
            np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[i+1])
        )
        similarities.append(sim)

    return similarities

def determine_breakpoints(similarities, percentile=95):
    """
    Determine breakpoints using percentile threshold
    """
    if not similarities:
        return []

    threshold = np.percentile(similarities, percentile)
    breakpoints = []
    for i, sim in enumerate(similarities):
        if sim < threshold:
            breakpoints.append(i + 1)  # +1 because we want to break after current sentence

    return breakpoints

def create_chunks(sentences, breakpoints):
    """
    Create chunks based on breakpoints
    """
    if not sentences:
        return []

    chunks = []
    start = 0

    for breakpoint in breakpoints:
        if breakpoint > start:
            chunk = " ".join(sentences[start:breakpoint])
            chunks.append(chunk)
            start = breakpoint

    # Add remaining sentences as last chunk
    if start < len(sentences):
        chunk = " ".join(sentences[start:])
        chunks.append(chunk)

    return chunks

def semantic_chunking(text, percentile=95):
    """
    Complete semantic chunking pipeline
    """
    sentences = split_sentences(text)
    if len(sentences) <= 1:
        return [text]

    embeddings = compute_sentence_embeddings(sentences)
    similarities = calculate_cosine_similarity(embeddings)
    breakpoints = determine_breakpoints(similarities, percentile)
    chunks = create_chunks(sentences, breakpoints)

    return chunks

# Test semantic chunking
test_text = "นี่คือประโยคแรกมันพูดถึงเรื่องหนึ่งประโยคที่สองพูดถึงเรื่องที่เกี่ยวข้องแต่ประโยคที่สามเปลี่ยนหัวข้อไปคุยเรื่องอื่นมันไม่เกี่ยวข้องกับเรื่องเดิม"
chunks = semantic_chunking(test_text)
print("Test chunks:", chunks)

Test chunks: ['นี่คือประโยคแรกมันพูดถึงเรื่องหนึ่งประโยคที่สองพูดถึงเรื่องที่เกี่ยวข้องแต่ประโยคที่สามเปลี่ยนหัวข้อไป คุยเรื่องอื่นมันไม่เกี่ยวข้องกับเรื่องเดิม']


In [21]:
# Cell 4: Process All Strategies into Chunks
def process_all_strategies(df):
    """
    Process all strategies into chunks
    """
    if df is None:
        return []

    all_chunks = []

    for idx, row in df.iterrows():
        strategy_name = row['ชื่อกลยุทธ์']

        # Combine description and example (including new Go example)
        full_text = f"{row['ลักษณะ']} {row['ตัวอย่างกลยุทธ์']}"

        # Add Go example if available
        if 'ตัวอย่างการเดินหมาก' in row and pd.notna(row['ตัวอย่างการเดินหมาก']):
            full_text += f" {row['ตัวอย่างการเดินหมาก']}"

        # Create chunks
        chunks = semantic_chunking(full_text)

        # Add metadata to chunks
        for chunk in chunks:
            all_chunks.append({
                'strategy_name': strategy_name,
                'category': row['หมวด'],
                'chunk_text': chunk,
                'full_text': full_text,
                'go_example': row.get('ตัวอย่างการเดินหมาก', '') if pd.notna(row.get('ตัวอย่างการเดินหมาก', '')) else ''
            })

    return all_chunks

# Process all strategies
all_chunks = process_all_strategies(df)
print(f"Total chunks created: {len(all_chunks)}")
if all_chunks:
    for i, chunk in enumerate(all_chunks[:3]):  # Show first 3 chunks
        print(f"Chunk {i+1}: {chunk['strategy_name']} - {chunk['chunk_text'][:100]}...")


Total chunks created: 424
Chunk 1: ปิดฟ้าข้ามทะเล - กลยุทธ์นี้เกี่ยวข้องกับการทำสิ่งใดสิ่งหนึ่งอย่างเปิดเผยในขณะที่แอบเตรียมการอื่น ๆ มักจะเกิดขึ้นเมื่อ...
Chunk 2: ปิดฟ้าข้ามทะเล - ยิ่งเมื่อต้องโจมตีศัตรูที่แข็งแกร่งและเตรียมพร้อม หรือเมื่อเผชิญหน้ากับศัตรูที่ทรงพลังเป็นครั้งแรก ม...
Chunk 3: ปิดฟ้าข้ามทะเล - เกี่ยวข้องกับการ "ถอยทัพเสแสร้ง" โดยแกล้งทำตัวอ่อนน้อมและยอมจำนนต่อศัตรู ปล่อยให้พวกเขาลดความระมัดระ...


In [22]:
# Cell 5: Build FAISS Index
def build_faiss_index(chunks):
    """
    Build FAISS index for semantic retrieval
    """
    if not chunks:
        return None, None

    # Extract chunk texts
    chunk_texts = [chunk['chunk_text'] for chunk in chunks]

    # Encode all chunks
    embeddings = model.encode(chunk_texts)

    # Normalize embeddings for cosine similarity
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Build FAISS index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # Inner Product for normalized vectors
    index.add(embeddings.astype(np.float32))

    return index, embeddings

# Build FAISS index
faiss_index, chunk_embeddings = build_faiss_index(all_chunks)
if faiss_index is not None:
    print(f"FAISS index built with {faiss_index.ntotal} vectors")

FAISS index built with 424 vectors


In [23]:
# Cell 6: Semantic Retrieval Function
def semantic_retrieval(query, index, chunks, top_k=5):
    """
    Retrieve relevant chunks using FAISS
    """
    if index is None or not chunks:
        return []

    # Encode query
    query_embedding = model.encode([query])
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    # Search
    scores, indices = index.search(query_embedding.astype(np.float32), top_k)

    # Prepare results
    results = []
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        results.append({
            'chunk': chunks[idx],
            'score': float(score),
            'rank': i + 1
        })

    return results


In [24]:
# Cell 7: TF-IDF Re-Ranking
def tfidf_reranking(query, retrieved_chunks, alpha=0.7):
    """
    Re-rank retrieved chunks using TF-IDF + semantic scores
    """
    if not retrieved_chunks:
        return []

    # Prepare texts
    texts = [query] + [chunk['chunk']['chunk_text'] for chunk in retrieved_chunks]

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(texts)

    # Calculate TF-IDF similarities
    query_tfidf = tfidf_matrix[0]
    chunk_tfidf = tfidf_matrix[1:]

    tfidf_similarities = cosine_similarity(query_tfidf, chunk_tfidf)[0]

    # Combine scores
    reranked_results = []
    for i, result in enumerate(retrieved_chunks):
        semantic_score = result['score']
        tfidf_score = tfidf_similarities[i]

        combined_score = alpha * semantic_score + (1 - alpha) * tfidf_score

        reranked_results.append({
            'chunk': result['chunk'],
            'semantic_score': semantic_score,
            'tfidf_score': tfidf_score,
            'combined_score': combined_score,
            'original_rank': result['rank']
        })

    # Sort by combined score
    reranked_results.sort(key=lambda x: x['combined_score'], reverse=True)

    # Update ranks
    for i, result in enumerate(reranked_results):
        result['new_rank'] = i + 1

    return reranked_results

In [25]:
# Cell 8: Updated 6C Framework Definitions
# Updated 6C Framework in Thai
sixc_definitions = {
    'ความแข็งแกร่งในการรุก': (
        'ความสามารถในการสร้างและเปลี่ยนแปลงภูมิทัศน์เชิงกลยุทธ์อย่างมีประสิทธิภาพ\n'
        'Offensive Strength: ความสามารถในการกำหนดทิศทางและเป็นฝ่ายรุกก่อน\n'
        'ตัวอย่าง: เดินหมากที่ดูธรรมดาในกลางกระดาน แต่เป็นการเปิดโอกาสให้บุกเข้ายึดมุมภายหลัง'
    ),
    'ความแข็งแกร่งในการป้องกัน': (
        'ความสามารถในการตอบสนองต่อการกระทำหรือความท้าทายจากฝ่ายตรงข้ามได้อย่างมีประสิทธิภาพ\n'
        'Defensive Strength: ความสามารถในการตั้งรับและรักษาตำแหน่งเมื่อถูกโจมตี\n'
        'ตัวอย่าง: กดดันหมากตรงกลางให้แน่น แล้วถอยเข้ามาป้องกันกลุ่มที่ถูกล้อมไว้จนปลอดภัย'
    ),
    'ความสามารถในการสัมพันธ์': (
        'ความสามารถในการจัดการและใช้ประโยชน์จากความสัมพันธ์กับผู้มีส่วนได้ส่วนเสียภายนอก\n'
        'Relational Capacity: ความสามารถใช้หมากรอบข้างช่วยเสริมกำลังกัน\n'
        'ตัวอย่าง: ล่อให้คู่ต่อสู้มาตัด แล้วใช้หมากรอบข้างพารวมกันทำแต้มจากมุมอื่น'
    ),
    'ศักยภาพที่มี': (
        'การมีอยู่และการใช้ทรัพยากรอย่างมีกลยุทธ์\n'
        'Potential Energy: ศักยภาพหมากหรือพื้นที่ที่ยังไม่ได้ใช้อย่างเต็มที่\n'
        'ตัวอย่าง: กระโดดไปยังจุดว่าง เพื่อรอสร้างแนวหนาแล้วค่อยเคลื่อนหมากหลักเข้าสู่เกม'
    ),
    'ความพร้อมด้านเวลา': (
        'การใช้เวลาที่เหมาะสมในการตัดสินใจ\n'
        'Temporal Availability: การเลือกจังหวะที่เหมาะสมในการเดินหมาก\n'
        'ตัวอย่าง: ฉวยโอกาสเดินหมากกลางกระดานขณะคู่ต่อสู้กำลังสับสน เพื่อเก็บแต้มก่อน'
    ),
    'ความเหมาะสมในบริบท': (
        'ระดับที่การตัดสินใจสอดคล้องกับบริบทเชิงกลยุทธ์ ทำให้มีข้อมูลที่ถูกต้องและเกี่ยวข้อง\n'
        'Contextual Fit: ความเหมาะสมของการเดินหมากกับภาพรวมสถานการณ์\n'
        'ตัวอย่าง: ยั่วให้คู่ต่อสู้รับที่มุมบนก่อน แล้วย้ายหมากไปบุกมุมขวาล่างทันที'
    ),
}

def compute_6c_embeddings():
    """
    Compute embeddings for 6C definitions
    """
    definitions = list(sixc_definitions.values())
    embeddings = model.encode(definitions)
    return embeddings

def compute_6c_vector(text):
    """
    Compute 6C vector for given text
    """
    # Get 6C embeddings
    sixc_embeddings = compute_6c_embeddings()

    # Encode text
    text_embedding = model.encode([text])

    # Compute raw scores (dot product)
    raw_scores = np.dot(text_embedding, sixc_embeddings.T)[0]

    # Normalize to distribution
    distribution = np.exp(raw_scores) / np.sum(np.exp(raw_scores))

    return distribution

# Test 6C vector computation
test_text = "ฉันต้องการหลอกลวงคู่แข่งให้เข้าใจผิด"
sixc_vector = compute_6c_vector(test_text)
sixc_labels = list(sixc_definitions.keys())

print("6C Vector for test text:")
for i, (label, score) in enumerate(zip(sixc_labels, sixc_vector)):
    print(f"{label}: {score:.4f}")


6C Vector for test text:
ความแข็งแกร่งในการรุก: 0.1591
ความแข็งแกร่งในการป้องกัน: 0.1616
ความสามารถในการสัมพันธ์: 0.1710
ศักยภาพที่มี: 0.1641
ความพร้อมด้านเวลา: 0.1709
ความเหมาะสมในบริบท: 0.1733


In [32]:
# Cell 9: Compute 6C Patterns for All Strategies
def compute_all_strategy_patterns(df):
    """
    Compute 6C patterns for all strategies
    """
    if df is None:
        return {}

    patterns = {}

    for idx, row in df.iterrows():
        strategy_name = row['ชื่อกลยุทธ์']
        full_text = f"{row['ลักษณะ']} {row['ตัวอย่างกลยุทธ์']}"

        # Add Go example if available
        if 'ตัวอย่างการเดินหมาก' in row and pd.notna(row['ตัวอย่างการเดินหมาก']):
            full_text += f" {row['ตัวอย่างการเดินหมาก']}"

        # Compute 6C vector
        sixc_vector = compute_6c_vector(full_text)
        patterns[strategy_name] = sixc_vector

    return patterns

# Compute patterns for all strategies
strategy_patterns = compute_all_strategy_patterns(df)

if strategy_patterns:
    print("6C Patterns for all strategies:")
    for strategy, pattern in list(strategy_patterns.items())[:]:
        print(f"\n{strategy}:")
        for i, (label, score) in enumerate(zip(sixc_labels, pattern)):
            print(f"  {label}: {score:.4f}")

6C Patterns for all strategies:

ปิดฟ้าข้ามทะเล:
  ความแข็งแกร่งในการรุก: 0.1737
  ความแข็งแกร่งในการป้องกัน: 0.1655
  ความสามารถในการสัมพันธ์: 0.1566
  ศักยภาพที่มี: 0.1673
  ความพร้อมด้านเวลา: 0.1631
  ความเหมาะสมในบริบท: 0.1739

ล้อมเว่ยช่วยจ้าว:
  ความแข็งแกร่งในการรุก: 0.1714
  ความแข็งแกร่งในการป้องกัน: 0.1613
  ความสามารถในการสัมพันธ์: 0.1583
  ศักยภาพที่มี: 0.1659
  ความพร้อมด้านเวลา: 0.1641
  ความเหมาะสมในบริบท: 0.1789

ยืมดาบฆ่าคน:
  ความแข็งแกร่งในการรุก: 0.1651
  ความแข็งแกร่งในการป้องกัน: 0.1603
  ความสามารถในการสัมพันธ์: 0.1672
  ศักยภาพที่มี: 0.1658
  ความพร้อมด้านเวลา: 0.1662
  ความเหมาะสมในบริบท: 0.1754

รอซ้ำยามเปลี้ย:
  ความแข็งแกร่งในการรุก: 0.1712
  ความแข็งแกร่งในการป้องกัน: 0.1597
  ความสามารถในการสัมพันธ์: 0.1537
  ศักยภาพที่มี: 0.1663
  ความพร้อมด้านเวลา: 0.1711
  ความเหมาะสมในบริบท: 0.1780

ชิงซ้ำตามไฟ:
  ความแข็งแกร่งในการรุก: 0.1672
  ความแข็งแกร่งในการป้องกัน: 0.1589
  ความสามารถในการสัมพันธ์: 0.1622
  ศักยภาพที่มี: 0.1686
  ความพร้อมด้านเวลา: 0.1695
  ความ

In [27]:
# Cell 10: KL Divergence Matching
def kl_divergence(p, q, epsilon=1e-10):
    """
    Compute KL divergence between two distributions
    D_KL(p || q) = sum(p * log(p / q))
    """
    # Add small epsilon to avoid log(0)
    q = q + epsilon
    p = p + epsilon

    # Renormalize
    p = p / np.sum(p)
    q = q / np.sum(q)

    return np.sum(p * np.log(p / q))

def match_strategies_kl(situation_text, strategy_patterns, top_k=3):
    """
    Match situation with strategies using KL divergence
    """
    if not strategy_patterns:
        return []

    # Compute 6C vector for situation
    situation_vector = compute_6c_vector(situation_text)

    # Compute KL divergence for each strategy
    kl_scores = []
    for strategy_name, strategy_vector in strategy_patterns.items():
        kl_score = kl_divergence(situation_vector, strategy_vector)
        kl_scores.append((strategy_name, kl_score))

    # Sort by KL divergence (lower is better)
    kl_scores.sort(key=lambda x: x[1])

    return kl_scores[:top_k]


In [29]:
# Cell 11: Structured Analysis Function (No LLM)
def generate_structured_analysis(situation, top_strategies, retrieved_chunks):
    """
    Generate structured analysis without LLM
    """
    # Compute 6C vector for situation
    situation_vector = compute_6c_vector(situation)

    # Prepare context chunks
    context_chunks = []
    for i, result in enumerate(retrieved_chunks[:3]):
        context_chunks.append(f"[ref{i+1}] {result['chunk']['strategy_name']}: {result['chunk']['chunk_text']}")

    context = "\n".join(context_chunks)

    # Prepare strategy ranking
    strategy_ranking = []
    for i, (strategy, kl_score) in enumerate(top_strategies):
        strategy_ranking.append(f"{i+1}. {strategy} (KL: {kl_score:.6f})")

    ranking_text = "\n".join(strategy_ranking)

    # Generate structured analysis
    analysis = f"""**PLAN:**
Step 1: วิเคราะห์องค์ประกอบหลักของสถานการณ์
Step 2: ประเมิน 6C Framework ในบริบทของสถานการณ์
Step 3: วิเคราะห์คะแนน KL Divergence และความหมาย
Step 4: เชื่อมโยงกับข้อมูลประวัติศาสตร์
Step 5: สรุปกลยุทธ์ที่แนะนำ

**STEP 1**
**Title:** วิเคราะห์องค์ประกอบหลักของสถานการณ์
**Summary:** ระบุตัวละคร เป้าหมาย และแรงจูงใจในสถานการณ์: "{situation}"

**STEP 2**
**Title:** ประเมิน 6C Framework ในบริบทของสถานการณ์
**Summary:** การประเมิน 6C Framework สำหรับสถานการณ์นี้:
"""

    # Add 6C analysis
    for i, (label, score) in enumerate(zip(sixc_labels, situation_vector)):
        level = "สูง" if score > 0.2 else "กลาง" if score > 0.15 else "ต่ำ"
        analysis += f"\n- {label}: {score:.4f} ({level})"

    analysis += f"""

**STEP 3**
**Title:** วิเคราะห์คะแนน KL Divergence และความหมาย
**Summary:** คะแนน KL ต่ำหมายถึงความคล้ายคลึงสูงระหว่างรูปแบบ 6C ของสถานการณ์กับกลยุทธ์
{ranking_text}

**STEP 4**
**Title:** เชื่อมโยงกับข้อมูลประวัติศาสตร์
**Summary:** ข้อมูลประวัติศาสตร์ที่เกี่ยวข้อง:
{context}

**STEP 5**
**Title:** สรุปกลยุทธ์ที่แนะนำ
**Summary:** ให้คำแนะนำกลยุทธ์ที่ครอบคลุมและปฏิบัติได้

**RESPONSE:**
### **การวิเคราะห์สถานการณ์:**
สถานการณ์: "{situation}"

องค์ประกอบหลัก: การแลกเปลี่ยนและการได้มาซึ่งทรัพยากรมนุษย์ที่มีคุณค่า ซึ่งเกี่ยวข้องกับการตัดสินใจเชิงกลยุทธ์ที่ต้องชั่งน้ำหนักระหว่างการสูญเสียและการได้รับ

### **การประเมิน 6C Framework:**
"""

    for i, (label, score) in enumerate(zip(sixc_labels, situation_vector)):
        level = "สูง" if score > 0.2 else "กลาง" if score > 0.15 else "ต่ำ"
        analysis += f"\n{i+1}. **{label}:** {score:.4f} ({level})"

    analysis += f"""

### **กลยุทธ์ที่แนะนำ:**
**อันดับ 1:** {top_strategies[0][0]} (KL: {top_strategies[0][1]:.6f})
- มีรูปแบบ 6C ที่สอดคล้องกับสถานการณ์มากที่สุด
- เหมาะสำหรับการจัดการความซับซ้อนและการตัดสินใจเชิงกลยุทธ์

**อันดับ 2:** {top_strategies[1][0]} (KL: {top_strategies[1][1]:.6f})
- เป็นกลยุทธ์สำรองที่มีความเหมาะสมสูง
- สามารถนำมาใช้ร่วมกับกลยุทธ์หลัก

**อันดับ 3:** {top_strategies[2][0]} (KL: {top_strategies[2][1]:.6f})
- เป็นทางเลือกที่ควรพิจารณาเป็นแผนสำรอง
- เหมาะสำหรับสถานการณ์ที่ซับซ้อนขึ้น

### **ข้อมูลสนับสนุนจากประวัติศาสตร์:**
{context}

### **คำแนะนำการประยุกต์ใช้:**
ควรพิจารณาใช้กลยุทธ์อันดับ 1 เป็นหลัก โดยเน้นการสร้างความสมดุลระหว่างการให้และการรับ
และเตรียมกลยุทธ์สำรองจากอันดับ 2-3 เพื่อรองรับสถานการณ์ที่อาจเปลี่ยนแปลง
การตัดสินใจควรคำนึงถึงผลกระทบระยะยาวและความสัมพันธ์กับฝ่ายต่างๆ"""

    return analysis

In [30]:
# Cell 12: Main Analysis Function
def analyze_situation(situation):
    """
    วิเคราะห์สถานการณ์โดยไม่ใช้ LLM
    """
    print(f"=== การวิเคราะห์สถานการณ์ ===")
    print(f"สถานการณ์: {situation}")
    print()

    # Step 1: Semantic Retrieval
    print("1. Semantic Retrieval...")
    retrieval_results = semantic_retrieval(situation, faiss_index, all_chunks, top_k=5)

    # Step 2: TF-IDF Re-ranking
    print("2. TF-IDF Re-ranking...")
    reranked_results = tfidf_reranking(situation, retrieval_results)

    # Step 3: 6C + KL Matching
    print("3. 6C + KL Matching...")
    top_strategies = match_strategies_kl(situation, strategy_patterns, top_k=3)

    # Step 4: Generate Structured Analysis
    print("4. Generating Structured Analysis...")
    structured_analysis = generate_structured_analysis(situation, top_strategies, reranked_results)

    print("\n=== ผลลัพธ์การวิเคราะห์ ===")
    print("Top 3 กลยุทธ์ที่ใกล้เคียงที่สุด:")
    for i, (strategy, kl_score) in enumerate(top_strategies):
        print(f"{i+1}. {strategy} (KL Divergence: {kl_score:.4f})")

    print("\nข้อความที่เกี่ยวข้อง (TF-IDF + Semantic):")
    for result in reranked_results[:3]:
        print(f"- {result['chunk']['strategy_name']} | คะแนนรวม: {result['combined_score']:.4f}")
        print(f"  ข้อความ: {result['chunk']['chunk_text'][:100]}...")
        print()

    print("\n=== การวิเคราะห์แบบมีโครงสร้าง ===")
    print(structured_analysis)

    return {
        'situation': situation,
        'retrieved_chunks': reranked_results,
        'top_strategies': top_strategies,
        'structured_analysis': structured_analysis
    }

# Test the system
if df is not None and faiss_index is not None:
    test_situation = "จูกัดเหลียงที่พึงพอใจฝีมือเกียงอุยจึงอยากได้ตัวไว้ จึงยอมเสียแฮหัวหลิมซึ่งมีตำแหน่งเป็นถึงบุตรเขยของโจยอยเพียงเพื่อให้ได้มาซึ่งนายทหารที่มีสติปัญญาเป็นเลิศ"
    results = analyze_situation(test_situation)
else:
    print("Cannot run test - missing data or index")

=== การวิเคราะห์สถานการณ์ ===
สถานการณ์: จูกัดเหลียงที่พึงพอใจฝีมือเกียงอุยจึงอยากได้ตัวไว้ จึงยอมเสียแฮหัวหลิมซึ่งมีตำแหน่งเป็นถึงบุตรเขยของโจยอยเพียงเพื่อให้ได้มาซึ่งนายทหารที่มีสติปัญญาเป็นเลิศ

1. Semantic Retrieval...
2. TF-IDF Re-ranking...
3. 6C + KL Matching...
4. Generating Structured Analysis...

=== ผลลัพธ์การวิเคราะห์ ===
Top 3 กลยุทธ์ที่ใกล้เคียงที่สุด:
1. หลีตายแทนถาว (KL Divergence: 0.0002)
2. ยืมดาบฆ่าคน (KL Divergence: 0.0002)
3. ซ่อนดาบบนรอยยิ้ม (KL Divergence: 0.0002)

ข้อความที่เกี่ยวข้อง (TF-IDF + Semantic):
- โยนกระเบื้องล่อหยก | คะแนนรวม: 0.7999
  ข้อความ: ฝีมือเกียงอุยจึงอยากได้ตัวไว้ จึงยอมเสียหัวหลิมซึ่งมีตำแหน่งเป็นถึงบุตรเขยของโจโฉเพียงเพื่อให้ได้มาซ...

- ตีเรืออับปาง | คะแนนรวม: 0.4044
  ข้อความ: นำเอากลยุทธ์ตีเรืออับปางไปใช้ได้แก่ ขงเบ้งที่วางกลอุบายให้เตงงายหลงเชื่อว่าเป็นบุตรของจูกัดเกี๋ยม เพ...

- มีในไม่มี | คะแนนรวม: 0.3868
  ข้อความ: จึงฉวยโอกาสที่กำลังทหารของโจโฉกำลังเศร้าโศกเสียใจและไว้ทุกข์ให้แก่โจโฉ นำทัพไปเพื่อหวังตีทัพวุยก๊กแล...


=== การวิเค